In [4]:
import os
import transformers
import torch

os.chdir('/home2/kkms4641/GenerativeModel_Tobigs_Conference_20-21/')

!nvidia-smi

Sat Jul 13 17:22:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:34:00.0 Off |                  Off |
| 30%   29C    P8             16W /  300W |       2MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
user_query = "노을과 함께 해변에 있는 강아지 프로필 사진 만들어줘" #ex. 우리 강아지한테 한복을 입혀 줘.

In [6]:
# 원하는 경로에 다운 받았을 경우
model_id = "./Meta-Llama-3-8B-Instruct"

# 별도의 경로 지정 하지 않았을 경우 
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
  "text-generation",
  model=model_id,
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

reasoning_prompt = [
    {"role": "system", 
     "content": 
     """You are a prompt engineer who makes prompts for generative AI models. """},
    
    {"role": "user", 
     "content": 
     f"""### instruction ###
      I want to make an AI pet profile with a picture of my dog. 
      The given Input Query Text is the form of the AI pet profile that the user wants. 
      So what do you think are the conditions for a good Prompt to go through a generative AI model and create a good result image?
      and If Input Query is Korean language, Please translate Korean to English.
      Please make sure the prompt includes the word 'dog' exactly four time. not others.
      The word 'dog' should not appear consecutively.
      Finally, Please ensure the generated prompt does not exceed the maximum token length allowed by the CLIP tokenizer.
      The maximum token length for the CLIP tokenizer is 60 tokens.
     """}
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

pipeline.model = pipeline.model.half().to(torch.float32)

reasoning_outputs = pipeline(
                    reasoning_prompt,
                    eos_token_id=terminators,
                    do_sample=True,
                    temperature=0.6,
                    top_p=0.9,
                )

conditions = reasoning_outputs[0]["generated_text"][-1]['content']

prompt = [
    {"role": "system", 
     "content": 
     """You are a prompt engineer who makes prompts for generative AI models. 
        Your answer should always be prompt except explanation.
        All prompt factors should be separated by comma(like a, b, c)."""},
    
    {"role": "user", 
     "content": 
     f"""### instruction ###
    I want to make an AI pet profile with a picture of my dog. 
      The given Input Query Text is the form of the AI pet profile that the user wants. 
      So what do you think are the conditions for a good Prompt to go through a generative AI model and create a good result image?
      and If Input Query is Korean language, Please translate Korean to English.
      Please make sure the output prompt includes the word 'dog' exactly four time. 
      The word 'dog' should not appear consecutively.
      Finally, Please ensure the generated prompt does not exceed the maximum token length allowed by the CLIP tokenizer.
      The maximum token length for the CLIP tokenizer is 60 tokens.
      ### Input Text ### 
      Query: {user_query} 
      Conditions: {conditions}
      
      =========
      New Prompt: 
     """}
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    max_new_tokens=60,
    top_p=0.9,
)

answer = outputs[0]["generated_text"][-1]['content']

print("최종 입력 프롬프트 : ", answer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


최종 입력 프롬프트 :  Create a vibrant AI pet profile with a picture of my dog, showcasing a lively and adventurous dog surrounded by a serene beach scene at sunset. The profile should feature a sweet and affectionate dog in the foreground, with a soft and fluffy coat, and a calm and peaceful dog-themed landscape in the background


In [7]:
print(answer)

Create a vibrant AI pet profile with a picture of my dog, showcasing a lively and adventurous dog surrounded by a serene beach scene at sunset. The profile should feature a sweet and affectionate dog in the foreground, with a soft and fluffy coat, and a calm and peaceful dog-themed landscape in the background


In [8]:
command_solb = f"python test_pti_solb_arg.py --prompt \"{answer}\""
!{command_solb}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pipeline components...: 100%|███████████| 7/7 [00:00<00:00, 1167.87it/s]
LoRA : Patching Unet
LoRA : Patching text encoder1
LoRA : Patching text encoder2
LoRA : Patching token input
<s1>
<s1>_1
<s1>_2
Create a vibrant AI pet profile with a picture of my <s1> <s1>_1, showcasing a lively and adventurous <s1> <s1>_1 surrounded by a serene beach scene at sunset. The profile should feature a sweet and affectionate <s1> <s1>_1 in the foreground, with a soft and fluffy coat, and a calm and peaceful <s1> <s1>_1-themed landscape in the background
Token indices sequence length is longer than the specified maximum sequence length for this model (90 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['<s1>_1 <s1>_2 <s1> <s1>_1 <s1>_2 _ 1 - themed landscape in the background']
Token indices sequence length is longer than the specified maximum sequence length for 

In [8]:
command_jaemin = f"python test_pti_jaemin_arg.py --prompt \"{answer}\""
!{command_jaemin}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pipeline components...: 100%|███████████| 7/7 [00:00<00:00, 1175.96it/s]
LoRA : Patching Unet
LoRA : Patching text encoder1
LoRA : Patching text encoder2
LoRA : Patching token input
<s1>
<s1>_1
Create a pet profile with a picture of a happy <s1> <s1>_1, featuring a <s1> <s1>_1 wearing a traditional Korean dress, sitting on a green grass, with a cute expression, surrounded by a few <s1> <s1>_1 toys, showcasing a playful and friendly <s1> <s1>_1, with a <s1> <s1>_1's love for its human family, and a loyal <s1> <s1>_1
Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['love for its human family, and a loyal <s1> <s1>_1 <s1> <s1>_1 _ 1']
Token indices sequence length is longer than the specified maximum sequence length for this model (116 > 77). Running